## Preparing a dataset for centre back model
Aim is to collect a dataset on match lineups (defenders specifically) and match stats, specifically xG, goals and also touches in box / final 3rd.

### 1. Set up / dependencies

In [25]:
from statsbombpy import sb
import numpy as np
import pandas as pd

### 2. Read in data

In [27]:
# get the competition / season_ids
comps = sb.competitions()
comps = comps[comps.competition_id == 37]
comps.season_id

credentials were not supplied. open data access only


15    90
16    42
17     4
Name: season_id, dtype: int64

In [32]:
# get all the match ids
matches1 = sb.matches(competition_id=37, season_id=4)
matches2 = sb.matches(competition_id=37, season_id=42)
matches3 = sb.matches(competition_id=37, season_id=90)
matches = pd.concat([matches1, matches2, matches3])
match_ids = matches.match_id.values

credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only


In [46]:
# get the lineup data
lineups = pd.DataFrame()
for m in match_ids:
    lineup = sb.lineups(match_id=m)
    teams = list(lineup.keys())
    team1 = teams[0]
    team2 = teams[1]
    lineup1 = pd.DataFrame(lineup[team1])
    lineup2 = pd.DataFrame(lineup[team2])
    lineup1["team"] = team1
    lineup2["team"] = team2  
    lineup1["match_id"] = m
    lineup2["match_id"] = m
    lineups = pd.concat([lineups, lineup1, lineup2])

credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data acces

credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data acces

credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only


In [77]:
lineups["positions"].apply(lambda x: x[0] if len(x) > 0 else None)

0     {'position_id': 5, 'position': 'Left Center Ba...
1     {'position_id': 24, 'position': 'Left Center F...
2     {'position_id': 15, 'position': 'Left Center M...
3     {'position_id': 1, 'position': 'Goalkeeper', '...
4     {'position_id': 22, 'position': 'Right Center ...
                            ...                        
15    {'position_id': 2, 'position': 'Right Back', '...
16    {'position_id': 1, 'position': 'Goalkeeper', '...
17                                                 None
18                                                 None
19                                                 None
Name: positions, Length: 10860, dtype: object

In [81]:
# extract key info to new df columns
lineups["position"] = lineups["positions"].apply(lambda x: x[0]["position"] if len(x) > 0 else None)
lineups["start_time"] = lineups["positions"].apply(lambda x: x[0]["from"] if len(x) > 0 else None)
lineups["end_time"] = lineups["positions"].apply(lambda x: x[0]["to"] if len(x) > 0 else None)

In [92]:
# only really interested in centre backs
print(lineups.position.unique())
cbs = ["Left Center Back", "Right Center Back", "Center Back"]
cb_lineups = lineups[lineups.position.isin(cbs)]

['Left Center Back' 'Left Center Forward' 'Left Center Midfield'
 'Goalkeeper' 'Right Center Forward' 'Right Center Back' 'Center Midfield'
 'Center Back' 'Right Center Midfield' 'Left Midfield' 'Center Forward'
 'Right Midfield' 'Left Wing' 'Right Back' 'Left Back' 'Right Wing'
 'Center Attacking Midfield' None 'Left Defensive Midfield'
 'Right Defensive Midfield' 'Center Defensive Midfield'
 'Left Attacking Midfield' 'Right Attacking Midfield' 'Secondary Striker'
 'Right Wing Back' 'Left Wing Back']


In [93]:
cb_lineups.shape

(1476, 12)

In [94]:
# get event data related to matches
events = sb.events(match_id=match_ids[0])

credentials were not supplied. open data access only


In [96]:
events.shape

(3793, 35)

In [97]:
events.sample(5)

,bad_behaviour,ball_receipt,ball_recovery,block,carry,counterpress,dribble,duel,duration,foul_committed,...,possession_team,related_events,second,shot,substitution,tactics,team,timestamp,type,under_pressure
1619,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Manchester City WFC,[e90a6c4e-3d37-49a5-a321-6370d64a7be0],57,NaN,NaN,NaN,Chelsea FCW,00:13:57.438,Ball Receipt*,NaN
1447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Chelsea FCW,[17171c16-be0b-4779-b7ea-53033c6c7942],18,NaN,NaN,NaN,Chelsea FCW,00:45:18.980,Ball Receipt*,NaN
1269,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Chelsea FCW,[e44814a7-fc72-40a1-99d7-49bc5451b478],30,NaN,NaN,NaN,Chelsea FCW,00:27:30.860,Ball Receipt*,NaN
2978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.04,NaN,...,Manchester City WFC,[9caf68d8-8c73-4cb2-8f96-fa5ceda82fce],45,NaN,NaN,NaN,Chelsea FCW,00:42:45.260,Pressure,NaN
3743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Manchester City WFC,[c67957b8-63c7-49c2-914f-18ff53fc8a57],12,NaN,NaN,NaN,Chelsea FCW,00:24:12.411,Foul Committed,NaN


### 3. Clean data